In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
#from bs4 import BeautifulSoup
from collections import defaultdict
#import requests
%matplotlib inline
from sklearn.svm import SVC
from nltk.stem.snowball import SnowballStemmer

## remove special symbol
def rm_sym(df):
    df['review'] = df['review'].str.replace("&#039;",'\'')
    df['review'].head()
    df['rating_cate'] = ''
    df.loc[df['rating'] >= 7,'rating_cate'] = 'high'
    df.loc[df['rating'] <= 4,'rating_cate'] = 'low'
    df.loc[(df['rating'] > 4) & (df['rating'] < 7),'rating_cate'] = 'medium'
    return df

def clean_text(df_tem3):
    df_tem3['review'] = df_tem3['review'].str.replace("\"","").str.lower()
    df_tem3['review'] = df_tem3['review'].str.replace( r"(\\r)|(\\n)|(\\t)|(\\f)|(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(&#039;)|(\d\s)|(\d)|(\/)","")
    df_tem3['review'] = df_tem3['review'].str.replace("\"","").str.lower()
    df_tem3['review'] = df_tem3['review'].str.replace( r"(\$)|(\-)|(\\)|(\s{2,})"," ")
    df_tem3['review'].sample(1).iloc[0]

    stemmer = SnowballStemmer('english')
    df_tem3['review'] = df_tem3['review'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split(" ")]))
    return df_tem3

## input train and test data

In [ ]:
df = pd.read_csv('drugsCom_raw/drugsComTrain_raw.tsv',sep='\t',index_col=0)#.sample(40000)
df = rm_sym(df)
df_tem3 = df

test = pd.read_csv("drugsCom_raw/drugsComTest_raw.tsv",sep='\t', index_col=0)
test = rm_sym(test)

df_tem3 = clean_text(df_tem3)
test = clean_text(test)

In [ ]:
df_tem3.groupby('rating_cate').size()

In [ ]:
print(df_tem3.shape)
print(test.shape)

# Tensorflow

In [ ]:
import tensorflow as tf
import tensorflow 

#from tensorflow import tensorflow.keras

#from keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D 
from tensorflow.keras.layers import Dropout


# fix random seed for reproducibility

MAX_NB_WORDS = 500
max_review_length = 500
EMBEDDING_DIM = 160


## Text to sequence

In [ ]:
# Tokenize the data
tokenizer = Tokenizer(num_words = MAX_NB_WORDS, 
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',
                      lower=True, split=' ', char_level=False, 
                      oov_token=None, document_count=0)

tokenizer.fit_on_texts(df_tem3['review'])
train_sequences = tokenizer.texts_to_sequences(df_tem3['review'])
test_sequences = tokenizer.texts_to_sequences(test['review'])

In [ ]:
# truncate and pad input sequences
X_train = sequence.pad_sequences(train_sequences, maxlen=max_review_length)
X_test = sequence.pad_sequences(test_sequences, maxlen = max_review_length)


In [ ]:
# transform y to get_dummies
y_train = pd.get_dummies(df_tem3['rating_cate'])
y_test = pd.get_dummies(test['rating_cate'])

word_index = tokenizer.word_index
y_train.head()

In [ ]:
# Print shapes of data. 

print(X_train.shape, '<-- shape of train_data ready for val/train split.')
print(X_test.shape, '<-- shape of final_test_data ready for fedding to network.')
print(len(tokenizer.word_index), '<-- Length of Word Index')

## Training & Validation Data

In [ ]:
# Split Training & Validation Data
from sklearn.model_selection import train_test_split


print('creating train and validation data by dividing train_data in 80:20 ratio')
######################################################

X_train_t, X_train_val, Y_train_t, y_train_val = train_test_split(X_train, y_train,test_size = 0.2)

######################################################
print('train data shape:', X_train_t.shape)
print('validation data shape:', X_train_val.shape)
print('Data is ready for training!!')

## set up Model Sequential

In [ ]:
nb_words  = min(MAX_NB_WORDS, len(word_index))
lstm_out = max_review_length

model = Sequential()
model.add(Embedding(nb_words,EMBEDDING_DIM,input_length=max_review_length))
model.add(Dropout(0.2))

## add conv using kernal No.32 and size 3x3, actiation='relu'(rm neg)
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(LSTM(50))
model.add(Dropout(0.2))

#model.add(Attention(MAX_SEQUENCE_LENGTH))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

## fit the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Run LSTM Model
batch = 32 
epoch = 40

## set name for the mdoel
training_cycle = 1
notebookname = "Drug_Data_"
variant = "LSTM_w_stopwords_"
version = "1.0_"
title = notebookname + variant + version

stamp = '{}trainging_cycle{}batchsize_{}'.format(title,training_cycle,batch)
print(stamp)

## save the best model
best_model_path = title + stamp + 'best.h5'
model_checkpoint = ModelCheckpoint(best_model_path, save_best_only = True) ## save only best model

## if 4 steps without decreasing of loss in valid set, stop the trainning
early_stopping = EarlyStopping(patience = 4)

LSTM_model = model.fit(X_train_t, Y_train_t, batch_size=batch, epochs=epoch,
                       validation_data=(X_train_val, y_train_val),callbacks=[early_stopping], shuffle = True)

best_score = min(LSTM_model.history['val_loss'])

## why difference between train and val loss could be randomly different? local optima?
## add dropout to avoid overfitting

In [ ]:
plt.plot(LSTM_model.history['loss'],label='train')
plt.plot(LSTM_model.history['val_loss'],label='validation')
plt.ylabel("loss")
plt.xlabel("epoch")

In [ ]:
LSTM_model.history['val_acc']

In [ ]:
## first try
## 1000
[0.7633334,
 0.765,
 0.7733334,
 0.7766667,
 0.79,
 0.78333336,
 0.77833337,
 0.77833337,
 0.78666663]

## 10000
[0.7896667,
 0.78500015,
 0.79650015,
 0.7953333,
 0.7994998,
 0.79933333,
 0.79466665,
 0.79899997,
 0.79566664,
 0.7911668,
 0.8008332,
 0.8033334,
 0.7893331,
 0.8063332,
 0.80533326,
 0.8041667]

## 20000
[0.8406666798591613,
 0.8463333468437195,
 0.8442500100135804,
 0.8465000066757202,
 0.845666675567627,
 0.8399166750907898]

## 40000
[0.846000010728836,
 0.8528333415985108,
 0.8522500121593475,
 0.8555416769981384,
 0.829625009059906,
 0.8536250035762787,
 0.8491666755676269]

## all
[0.8746228654859085,
 0.8851105690445873,
 0.889078329655966,
 0.8945443354995075,
 0.896011581290825,
 0.8991320621279996,
 0.8992973819550564,
 0.9017668994849171,
 0.9046290611673865,
 0.908173181695814]


## add convential layer and dropout(0.2)
## 20000
[0.8463333458900452,
 0.8495833463668824,
 0.8557500090599061,
 0.8566666769981385,
 0.858500009059906,
 0.8403333406448364,
 0.8505833492279052,
 0.8565000033378601]

## 40000
[0.8470000097751618,
 0.8522500097751617,
 0.8575416767597198,
 0.8592500100135804,
 0.8623750069141388,
 0.8616666796207428,
 0.861833342552185,
 0.86329167842865]

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accr = model.evaluate(X_test,y_test, batch_size = 100)
accr

In [ ]:
## 10000
## [0.4519780118446444, 0.8386464840319391]

## 40000
[0.3983093709805641, 0.8546789216107478]

In [ ]:
y_test_pred = model.predict_classes(X_test)

In [ ]:
y_test#.values#.argmax(axis=1)

In [ ]:
accuracy_score(y_test.values.argmax(axis=1),y_test_pred)
## 10000
## 0.7498047092958375

## 40000
## 0.7756574787040137

## second try
## 40000
## 0.7865007625637019

In [ ]:
confusion_matrix(y_test.values.argmax(axis=1),y_test_pred)

In [ ]:
y_test.sum()